In [16]:
import numpy as np
import pandas as pd

In [17]:
# read the csv file
raw_data = pd.read_csv("NYPD_Complaint_Data_Historic_short.csv")

/Users/zexichen/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
raw_data[:10]

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",...,44.0,INSIDE,BAR/NIGHT CLUB,NaN,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)"
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,103.0,OUTSIDE,NaN,NaN,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)"
2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,28.0,NaN,OTHER,NaN,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,105.0,INSIDE,RESIDENCE-HOUSE,NaN,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)"
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,13.0,FRONT OF,OTHER,NaN,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)"
5,365661343,12/31/2015,23:18:00,12/31/2015,23:25:00,12/31/2015,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,71.0,FRONT OF,DRUG STORE,NaN,NaN,996149.0,181562.0,40.665023,-73.957111,"(40.665022689, -73.957110763)"
6,608231454,12/31/2015,23:15:00,NaN,NaN,12/31/2015,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",...,7.0,OPPOSITE OF,STREET,NaN,NaN,987373.0,201662.0,40.720200,-73.988735,"(40.720199996, -73.988735082)"
7,265023856,12/31/2015,23:15:00,12/31/2015,23:15:00,12/31/2015,118,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,...,46.0,FRONT OF,STREET,NaN,NaN,1009041.0,247401.0,40.845707,-73.910398,"(40.845707148, -73.910398033)"
8,989238731,12/31/2015,23:15:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,48.0,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1014154.0,251416.0,40.856711,-73.891900,"(40.856711291, -73.891899956)"
9,415095955,12/31/2015,23:10:00,12/31/2015,23:10:00,12/31/2015,341,PETIT LARCENY,338.0,"LARCENY,PETIT FROM BUILDING,UN",...,19.0,INSIDE,DRUG STORE,NaN,NaN,994327.0,218211.0,40.765618,-73.963623,"(40.765617688, -73.96362342)"


In [19]:
lu = {'cfd':'CMPLNT_FR_DT', 'cft':'CMPLNT_FR_TM', 'ctd':'CMPLNT_TO_DT', 'ctt':'CMPLNT_TO_TM', 'lcc':'LAW_CAT_CD', 'apc':'ADDR_PCT_CD'}
law_cat = {'FELONY':1, 'MISDEMEANOR':2, 'N.Y. HOUSING POLICE':3}
effective_columns = ['CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'LAW_CAT_CD', 'ADDR_PCT_CD', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude']

In [20]:
# count nan values in each column
def count_nan(df):
    return df.isnull().sum()

In [21]:
print raw_data.shape

(2979512, 24)


In [22]:
def print_num_nan_rows_per_column(df):
    for col in df:
        print 'The number of nan rows in column %s is %d'%(col, count_nan(df[col]))

In [23]:
def replace_null(df_to_replace, df_from_replace):
    rows = df_to_replace.isnull() & (~df_from_replace.isnull())
    df_to_replace[rows] = df_from_replace[rows]
    return

In [24]:
def remove_rows(df, col):
    # remove rows if it is nan in some column of the dataframe
    df.drop(df.index[df[col].isnull()], inplace=True)
    
    return

In [25]:
# pre-process event occurs date. First, if the date does not exist, using event end date to replace it.
# Afterwards, remove null rows
replace_null(raw_data[lu['cfd']], raw_data[lu['ctd']])
remove_rows(raw_data, lu['cfd'])

/Users/zexichen/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [26]:
# pre-process event occurs time. First, if the time does not exist, using event end time to replace it.
# Afterwards, remove null rows
replace_null(raw_data[lu['cft']], raw_data[lu['ctt']])
remove_rows(raw_data, lu['cft'])

/Users/zexichen/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [27]:
# replace the different crime categories with numerial values
raw_data[lu['lcc']] = pd.Categorical(raw_data[lu['lcc']]).codes

In [28]:
remove_rows(raw_data, lu['apc'])

In [29]:
# extract useful columns
new_data = raw_data[effective_columns ]

In [30]:
new_data.to_csv('preprocessed_NYPD_Complaint_Data_Historic_short.csv', index=False)

In [31]:
# read the csv file
raw_data_1 = pd.read_csv("preprocessed_NYPD_Complaint_Data_Historic_short.csv")

In [32]:
print_num_nan_rows_per_column(raw_data_1)

The number of nan rows in column CMPLNT_FR_DT is 0
The number of nan rows in column CMPLNT_FR_TM is 0
The number of nan rows in column LAW_CAT_CD is 0
The number of nan rows in column ADDR_PCT_CD is 0
The number of nan rows in column X_COORD_CD is 105583
The number of nan rows in column Y_COORD_CD is 105583
The number of nan rows in column Latitude is 105583
The number of nan rows in column Longitude is 105583
